In [187]:
# Count type 2

In [190]:
from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

In [191]:
import lltk
C=lltk.load('CanonFiction')

In [192]:
%%write_and_run .count.slingshot.v2.py

### SLINGSHOT

import sys
sys.path.append('../..')
from abslithist.words import *
import ujson as json

dfnorms=get_allnorms()

def score_freqs(inpd):
    path_freqs=inpd['path_freqs']
    try:
        with open(path_freqs) as f: freqs=json.load(f)
    except Exception:
        return []
    freql=[w for w,c in freqs.items() for i in range(c)]
    freqdf=pd.DataFrame(index=freql)
    
    # by col
    odf=pd.DataFrame()
    for col in dfnorms.columns:
        scol=dfnorms[col]
        scoldf=freqdf.join(scol.dropna().to_frame(),how='inner')
        # make avg
        avgdf=pd.DataFrame(dict((x,scoldf.agg(x)) for x in ['mean','median','sum','std','size']))
        odf=odf.append(avgdf)
    return odf.rename_axis('norm').reset_index().to_dict('records')

In [182]:
# score_freqs(C.t.path_freqs)

In [184]:
# prepare objects
objects = [
    {'id':t.id, 'path_freqs':t.path_freqs}
    for t in C.texts()
]

In [185]:
# score_freqs(objects[0])

In [186]:
# Slingshot!
import mpi_slingshot as sl
cmd = sl.shoot(
    func='score_freqs',
    path_src='.count.slingshot.v2.py',
    objects=objects,
    savedir='../../data/scores/v2',
#     overwrite=True
)
cmd

[Slingshot] Preparing input: 100%|██████████| 1809/1809 [00:00<00:00, 73952.92it/s]


'slingshot -code ../../data/scores/v2/func.py -func score_freqs -pathlist ../../data/scores/v2/input.jsonl -savedir ../../data/scores/v2 -resume -resume -nosave'

In [ ]:
!{cmd}